# The Gender Wage Gap and Part-time workers

In this project, we examine the gender wage gap in various sectors in Denmark and how it has developed over time. Many other studies have shown that education and work experience do not have an impact on the gender wage gap. Therefore, we will also look at the share of female part-time workers in these sectors and how it has changed over the years. Women may be more likely to work part-time than men, often due to caregiving responsibilities or other factors.

We will use data from Statistics Denmark to conduct our analysis and visualize the results. By doing so, we hope to gain insights into the trends and patterns of the gender wage gap and part-time work in different sectors and understand the factors that may contribute to these differences.

First we need to modules and get acces to Statistics Denmark´s API. 

In [ ]:
# The DST API wrapper
%pip install git+https://github.com/elben10/pydst

In [ ]:
import pandas as pd
import ipywidgets as widgets
from ipywidgets import interact, IntSlider
from ipywidgets import interact
import plotly.graph_objs as go
import plotly.graph_objects as go
import plotly.express as px

import warnings 
warnings.filterwarnings("ignore", category=RuntimeWarning)

# Import pydsta and setup data loader with the langauge 'english'
import pydst
dst = pydst.Dst(lang='en')

# 1. Read and Clean Data

To this project we uses data for Denmark from the Statistics Denmark's API. We uses data on wages (LONS30) and employment (RAS307). 

**1.1 Hourly Wage per Sector**

We retrieve data from the 'LONS30' statistic and store the variables in the wage_var variable. This statistic contains data on earnings by sex, sector, and region. Since we want the data in a wide format, we first handle the data for female wages and then for male wages.

We store the data on female hourly wages by sector for all possible years in wage_k. To select the variables to download, we use the variables argument, which is a dictionary where the keys are the variable names, and the values are lists of the values to select. By using {'KØN': ['K']}, we only select observations where the gender is female. We filter the data to include all salary earners in Denmark. We also choose to look at earnings in DKK per hour to facilitate wage comparisons between full-time and part-time workers.

As the focus of this project is limited, we do not use all the variables. Therefore, we drop some of them: 'OMRÅDE', 'AFLOEN', 'LONGRP', 'LØNMÅL', and 'KØN'. By using 'axis=1', we drop columns instead of rows. Even though we remove some variables, the rows from these variables remain, so we use groupby to aggregate the rows by year and sector.
Then, we rename the variables to have more meaningful names. For example, 'TID' is changed to 'Year'.

The same process is applied to the male hourly wage data to convert it to a wide format.

Finally, we merge the two DataFrames (wage_k and wage_m) based on the 'Year' and 'Sector' columns, so that the resulting DataFrame 'wage' contains columns for both female and male hourly wages. The how='left' argument means that we keep all rows from wage_k (which contains the female wages) and only add matching rows from wage_m (which contains the male wages). The resulting DataFrame has columns for 'Year', 'Sector', 'Female Hourly wage', and 'Male Hourly wage'.

In [ ]:
# Female wage

# Get variables for the LONS30 table
wage_vars = dst.get_variables(table_id='LONS30')

# Download data
wage_k = dst.get_data(table_id = 'LONS30', 
                      variables={'Tid':['*'],
                                 'KØN': ['K'], 
                                 'OMRÅDE': ['000'],
                                 'SEKTOR':['*'],
                                 'AFLOEN':['TIFA'],
                                 'LONGRP':['LTOT'],
                                 'LØNMÅL':['FORINKL']})

# Drop columns
drop = ['OMRÅDE', 'AFLOEN', 'LONGRP', 'LØNMÅL', 'KØN']
wage_k.drop(drop, axis=1, inplace=True)

# Rename columns
wage_k.rename(columns = {'TID':'Year', 
                         'SEKTOR': 'Sector',
                         'INDHOLD': 'Female Hourly wage'}, inplace=True)

In [ ]:
# Male wage

# Download data
wage_m = dst.get_data(table_id = 'LONS30', 
                      variables={'Tid':['*'],
                                 'KØN': ['M'], 
                                 'OMRÅDE': ['000'],
                                 'SEKTOR':['*'],
                                 'AFLOEN':['TIFA'],
                                 'LONGRP':['LTOT'],
                                 'LØNMÅL':['FORINKL']})

# Drop columns
drop = ['OMRÅDE', 'AFLOEN', 'LONGRP', 'LØNMÅL', 'KØN']
wage_m.drop(drop, axis=1, inplace=True)

# Rename columns
wage_m.rename(columns = {'TID':'Year', 
                         'SEKTOR': 'Sector',
                         'INDHOLD': 'Male Hourly wage'}, inplace=True)

In [ ]:
# Merge the wage_k and wage_m data frames based on the 'Year' and 'Sector' columns using a left join. 
wage=pd.merge(wage_k,wage_m,how='left',on=['Year', 'Sector'])

**1.2 Extent of Working Time**

This code downloads and cleans data related to employment in Denmark, with a specific focus on full-time and part-time work. The data is sourced from 'RAS307' provided by Statistics Denmark.

First, the code specifies the variables to include in the data and drops unnecessary columns (ARBOMFANG and KØN). Then, it renames the remaining columns (TID, SEKTOR, and INDHOLD) to 'Year', 'Sector', and 'Female Full-time', respectively. The data is sorted by year, and the resulting dataset is displayed.

Next, we repeat the code for part-time workers, but this time we set the variable 'ARBOMFANG' to 'D' to specify part-time work. The code then drops unnecessary columns and renames the remaining columns to 'Year', 'Sector', and 'Female Part-time', respectively.

The same process is applied for male workers.

Finally, the dataframes are merged on 'Year' and 'Sector' to create the 'time' dataframe. This merged dataframe contains the number of female and male employees, both full-time and part-time, in each sector for each year.

In [ ]:
# Female Full-time

# Download data
time_k_ft = dst.get_data(table_id = 'RAS307', 
                         variables={'TID':['*'], 
                                    'KØN':['K'], 
                                    'BRANCHE07': ['*'],
                                    'SEKTOR':['*'],
                                    'ARBOMFANG':['H']})

# Drop unnecessary columns
drop_cols = ['ARBOMFANG', 'KØN']
time_k_ft.drop(drop_cols, axis=1, inplace=True)

# Rename columns
time_k_ft.rename(columns={'TID': 'Year',
                          'SEKTOR': 'Sector',
                          'INDHOLD': 'Female Full-time'}, inplace=True)

# Group the data by 'Year' and 'Sector' and sum the values
time_k_ft = time_k_ft.groupby(['Year', 'Sector']).sum()

# Reset the index
time_k_ft.reset_index(inplace=True)

# Show first few rows
time_k_ft = time_k_ft.sort_values(by='Year', ascending=True)
time_k_ft.head(10)

## Female Part-time

# Download data
time_k_pt = dst.get_data(table_id = 'RAS307', 
                         variables={'TID':['*'], 
                                    'KØN':['K'], 
                                    'BRANCHE07': ['*'],
                                    'SEKTOR':['*'],
                                    'ARBOMFANG':['D']})

# Drop columns
drop = ['ARBOMFANG', 'KØN']
time_k_pt.drop(drop, axis=1, inplace=True)

# Rename columns
time_k_pt.rename(columns = {'TID':'Year', 
                            'SEKTOR': 'Sector',
                            'INDHOLD': 'Female Part-time'}, inplace=True)

# Group the data by 'Year' and 'Sector' and sum the values
time_k_pt = time_k_pt.groupby(['Year', 'Sector']).sum()

# Reset the index
time_k_pt.reset_index(inplace=True)

# Show first few rows
time_k_pt = time_k_pt.sort_values(by='Year', ascending=True)

In [ ]:
## Male Full-time

# Download data
time_m_ft = dst.get_data(table_id = 'RAS307', 
                         variables={'TID':['*'], 
                                    'KØN':['M'], 
                                    'BRANCHE07': ['*'],
                                    'SEKTOR':['*'],
                                    'ARBOMFANG':['H']})

# Drop columns
drop = ['ARBOMFANG', 'KØN']
time_m_ft.drop(drop, axis=1, inplace=True)

# Rename columns
time_m_ft.rename(columns = {'TID':'Year', 
                            'SEKTOR': 'Sector',
                            'INDHOLD': 'Male Full-time'}, inplace=True)

# Group the data by 'Year' and 'Sector' and sum the values
time_m_ft = time_m_ft.groupby(['Year', 'Sector']).sum()

# Reset the index
time_m_ft.reset_index(inplace=True)

# Show first few rows
time_m_ft = time_m_ft.sort_values(by='Year', ascending=True)
time_m_ft.head(10)

## Male Part-time

# Download data
time_m_pt = dst.get_data(table_id = 'RAS307', 
                         variables={'TID':['*'], 
                                    'KØN':['M'], 
                                    'BRANCHE07': ['*'],
                                    'SEKTOR':['*'],
                                    'ARBOMFANG':['D']})

# Drop columns
drop = ['ARBOMFANG', 'KØN',]
time_m_pt.drop(drop, axis=1, inplace=True)

# Rename columns
time_m_pt.rename(columns = {'TID':'Year', 
                            'SEKTOR': 'Sector',
                            'INDHOLD': 'Male Part-time'}, inplace=True)

# Group the data by 'Year' and 'Sector' and sum the values
time_m_pt = time_m_pt.groupby(['Year', 'Sector']).sum()

# Reset the index
time_m_pt.reset_index(inplace=True)

# Show first few rows
time_m_pt = time_m_pt.sort_values(by='Year', ascending=True)


In [ ]:
# Merge the time_k_ft and time_k_pt data frames based on the 'Year' and 'Sector' columns using a left join.
time_k=pd.merge(time_k_ft,time_k_pt,how='left',on=['Year', 'Sector'])

# Group the time_k data frame by the 'Year' and 'Sector' columns, and sum the values for each group.
time_k = time_k.groupby(['Year', 'Sector']).sum()

# Merge the time_m_ft and time_m_pt data frames based on the 'Year' and 'Sector' columns using a left join.
time_m=pd.merge(time_m_ft,time_m_pt,how='left',on=['Year', 'Sector'])

# Group the time_m data frame by the 'Year' and 'Sector' columns, and sum the values for each group.
time_m = time_m.groupby(['Year', 'Sector']).sum()

# Merge the time_k and time_m data frames based on the 'Year' and 'Sector' columns using a left join.
time=pd.merge(time_k,time_m,how='left',on=['Year', 'Sector'])


# 2. Exploring Data

**2.1 Hourly Wage by Gender and Sector**

We use the data that we downloaded and cleaned in section 1.1. We want to study how the wages have developeed over the years so we use the following code. 

The following code creates an interactive plot that shows the hourly wage of males and females for different sectors. Using the dropdown function, you can select the sector to examine.

In [ ]:
# Group the data by year, sector, and gender, and calculate the mean hourly wage
wage_by_year_sector = wage.groupby(['Year', 'Sector']).mean().reset_index()

# Create the plot layout
layout = go.Layout(xaxis=dict(title='Year'), yaxis=dict(title='Hourly wage (DKK)'))

# Define the dropdown options
sector_options = list(wage_by_year_sector['Sector'].unique())

# Define the callback function
def update_plot(sector):
    # Filter the data based on the selected sector
    filtered_data = wage_by_year_sector[wage_by_year_sector['Sector'] == sector]

    # Create traces for the full-time and part-time data
    trace_k = go.Scatter(x=filtered_data['Year'], y=filtered_data['Female Hourly wage'], mode='lines',line=dict(color='red', width=2), name='Female')
    trace_m = go.Scatter(x=filtered_data['Year'], y=filtered_data['Male Hourly wage'], mode='lines',line=dict(color='blue', width=2), name='Male')

    # Create the plot figure and add the traces and layout
    fig = go.Figure(data=[trace_k, trace_m], layout=layout)
    fig.update_layout(title='Average hourly wage by gender and sector ({})'.format(sector))
    fig.show()

# Create the interactive dropdown
interact(update_plot, sector=sector_options)

The graph depicts a proportional increase in hourly wages across all sectors, albeit with variations in the rate of increase and the initial and final wage levels. Notably, the female hourly wage started at 270.72 DKK and reached 316.83 DKK in 2021, whereas the male hourly wage increased from 309.21 DKK in 2013 to 352.15 DKK in 2021.

There are variations in hourly wages among different sectors. For instance, the 'Municipal government' sector shows a relatively narrow range of hourly wages. Conversely, the 'Regional government' sector displays a significantly wider disparity in hourly wages, with a broader range of initial and final wage levels

**2.2 Extent of Working Time by Gender and Sector**

In this section, we examine the gender differences in working time. Full-time employment is defined as working more than 32 hours per week, while part-time employment is defined as working 32 hours or less per week.

The code provided below calculates the number of employees for each year and sector. It creates an interactive plot that illustrates the number of female and male employees, differentiating between full-time and part-time employment. Additionally, a dropdown menu is included to select a specific sector of interest. The resulting plot displays the employment trends by gender within the chosen sector over the years. Separate lines are used to represent full-time employment (solid line) and part-time employment (dashdot line) for each gender.

In [ ]:
# Group the data by year, sector, and industry and calculate the mean for each group
time_k_avg = time_k.groupby(['Year', 'Sector']).sum().reset_index()
time_m_avg = time_m.groupby(['Year', 'Sector']).sum().reset_index()

# Create the plot layout
layout2 = go.Layout(xaxis=dict(title='Year'),
                   yaxis=dict(title='Number of employees', tickformat=',.0f'))

# Define the dropdown options
sector_options_time = list(time_k_avg['Sector'].unique())

# Create the figure
fig = go.Figure(layout=layout2)

# Define the callback function
def update_plot(sector):
    # Filter the data based on the selected sector
    filtered_data_k = time_k_avg[time_k_avg['Sector'] == sector]
    filtered_data_m = time_m_avg[time_m_avg['Sector'] == sector]

    # Create traces for the full-time and part-time data
    trace_k_ft = go.Scatter(mode='lines', x=filtered_data_k['Year'], y=filtered_data_k['Female Full-time'],  line=dict(color='red', width=2), name='Female Full-time')
    trace_k_pt = go.Scatter(mode='lines', x=filtered_data_k['Year'], y=filtered_data_k['Female Part-time'], line=dict(color='red', dash='dashdot'), name='Female Part-time')

    trace_m_ft = go.Scatter(mode='lines', x=filtered_data_m['Year'], y=filtered_data_m['Male Full-time'], line=dict(color='blue', width=2), name='Male Full-time')
    trace_m_pt = go.Scatter(mode='lines', x=filtered_data_m['Year'], y=filtered_data_m['Male Part-time'], line=dict(color='blue', dash='dashdot'), name='Male Part-time')

    # Create the plot figure and add the traces and layout
    fig = go.Figure(data=[trace_k_ft, trace_k_pt, trace_m_ft, trace_m_pt], layout=layout2)
    fig.update_layout(title='Work Time by gender and sector ({})'.format(sector))
    fig.show()

# Create the interactive dropdown
interact(update_plot, sector=sector_options_time)


The plot shows that most female and male workers work in private corporations. There is a significant difference between the number of full-time female workers and full-time male workers. In 2021, almost 900,000 male workers were working full-time in private corporations, whereas only 415,000 females were working full-time. The difference between the number of part-time workers is not significant.

In the municipal government sector, there are more part-time female workers than full-time male workers. There are 119,000 part-time female workers compared to only 30,600 part-time male workers.

Overall, the plot shows that the division of labor between genders varies greatly across sectors.

# 3. Merge data sets

The following code is used to merge two separate datasets that contain information on wages and employees. The code utilizes the inner merging function, which combines the datasets based on the intersection of matching values in specified columns. This merging function retains only the rows from each dataset that have matching values in the specified columns, while removing the rows that do not have matching values.

In [ ]:
# Merge the dataframes
merged = pd.merge(wage,time, how='inner', on=['Year', 'Sector'])

# 4.    Analysis

**4.1 Descriptive Statistics**

First we look at some descriptive statistics for the wages to examine the differences between the gender.

In [ ]:
# Defining descriptive stat
def describe_stats(year):
    print(f'Female and male average hourly wages in {year}')
    I = merged['Year'] == year
    display(merged[I].iloc[:, 2:4].describe().round(2))

slider = IntSlider(min=merged['Year'].min(), max=merged['Year'].max(), step=1, value=2013)
interact(describe_stats, year=slider);

Based on the table above, we can observe the summary statistics of hourly wages for females and males. By utilizing the slider, we discover that in 2013, the average hourly wage for women in the two sectors was 271.72 DKK, while for men it was 319.27 DKK. This indicates that, on average, women earn 85.11 percent of men's earnings in these sectors.

In 2021, the female hourly wage has increased to 321.05 DKK, while the male hourly wage is now 355.13 DKK. Notably, the female hourly wage has experienced a greater increase compared to the male wage. Consequently, in 2021, women on average earn 90.40 percent of men's earnings in these sectors.

**4.2 The Gender Wage Gap**

To determine if any specific sectors are contributing to the reduction in gender wage differences, we plot the gender wage gap over the years.

First, the code calculates the gender wage gap as a percentage by dividing the hourly wage of female workers by the hourly wage of male workers. This calculation allows us to quantify and analyze the disparity between earnings for each gender.

Next, we create a line plot to visually depict the gender wage gap over time for each sector. The x-axis represents the years, while the y-axis represents the gender wage gap. By examining this plot, we can observe the changes in wage disparity between genders across different sectors over the years.

In [ ]:
# Calculating the gender wage gap
merged['Gender wage gap'] = 100 - (merged['Female Hourly wage'] / (merged['Male Hourly wage'])) * 100

# Create a line plot of gender wage gap over the years for each sector using Plotly
fig = px.line(merged, x='Year', y='Gender wage gap', color='Sector', title='Gender Wage Gap by Sector Over Time')

# Format the y-axis labels to display only 2 decimal places
fig.update_layout(yaxis=dict(tickformat='.2f'))

# Add x and y axis labels
fig.update_xaxes(title='Year')
fig.update_yaxes(title='Gender Wage Gap (%)')

# Display the plot
fig.show()

The plot above illustrates the gender wage gap in the municipal and regional government sectors. The data indicates a significantly smaller wage gap between men and women in the municipal government sector compared to the regional government sector. Specifically, the graph demonstrates that the gender wage gap in the regional government sector was initially much higher, starting at 21.81 in 2013, but has steadily decreased over time. By 2021, the wage gap had reduced by almost 40 pct. to 13.64. A gender wage gap of 13.64 pct. means that, on average, female workers earn 13.64pct. less than their male counterparts in the same sector.

In contrast, the gender wage gap in the municipal government sector was 5.58 in 2013 and remained relatively stable throughout the period. In 2021, the gender wage gap in the municipal government sector was 4.54, which is significantly lower than the national average wage gap of approximately 15pct.

According to the graph presented in section 2.1, the wages of female employees in the regional government sector have increased at a higher rate than male wages. This finding supports the observation that the gender wage gap in this sector has decreased over time.

**4.3 Female Work and Working Time**

Next, we examine the proportion of female workers in the two sectors. Initially, we calculate the share of female workers as a percentage by summing the number of female full-time and part-time workers and dividing it by the total number of female workers.

Using the Plotly library, we create a line plot that visualizes the share of female workers over time for each sector. The x-axis represents the years, while the y-axis represents the percentage of female workers. Additionally, the plot employs color coding to differentiate between the different sectors.

In [ ]:
# Calculating the share of female workers
merged['Share of Female Workers'] = (merged['Female Full-time'] + merged['Female Part-time']) / (merged['Male Full-time'] + merged['Male Part-time'] + merged['Female Full-time'] + merged['Female Part-time']) * 100

# Create a line plot of gender wage gap over the years for each sector using Plotly
fig = px.line(merged,   x='Year', 
                        y='Share of Female Workers', 
                        color='Sector', 
                        title='Share of Female workers')

# Format the y-axis labels to display only 2 decimal places
fig.update_layout(yaxis=dict(tickformat='.2f'))

# Add x and y axis labels
fig.update_xaxes(title='Year')
fig.update_yaxes(title='Share of Female Workers (%)')

# Display the plot
fig.show()


The graph illustrates the percentage of female workers in the regional and municipal government sectors over time. Throughout the entire time period, the share of female workers was consistently higher in the regional government sector compared to the municipal government sector. However, there has been a slight decline in the female share over the years, reaching a minimum of 79.08 pct. in 2017.

In contrast, the female share in the municipal government sector has shown significant variation, with a peak of 77.96 pct. observed in 2015.

To get a better understanding of the female employment, we examine the female share of part-time workers which is calculated as female part-time workers divided by the total of female workers. We show the share in a line plot. 

In [ ]:
# Calculating the share of female part-time workers
merged['Share of Female Part-time Workers'] = (merged['Female Part-time']) / (merged['Female Full-time'] + merged['Female Part-time']) * 100

# Create a line plot of gender wage gap over the years for each sector using Plotly
fig = px.line(merged,   x='Year', 
                        y='Share of Female Part-time Workers', 
                        color='Sector', 
                        title='Share of Female Part-time Workers')

# Format the y-axis labels to display only 2 decimal places
fig.update_layout(yaxis=dict(tickformat='.2f'))

# Add x and y axis labels
fig.update_xaxes(title='Year')
fig.update_yaxes(title='Share of Female Part-time Workers (%)')

# Display the plot
fig.show()

The graph displays the share of female part-time workers in the Municipal and Regional Government sectors. It reveals that the highest share is observed in the Municipal Government sector, declining from 38.27 pct. to 31.78 pct. over the years, indicating a downward trend. Conversely, the share of female part-time workers in the Regional Government sector remains relatively stable, fluctuating between 19.43 pct. and 22.68 pct. during the same period.

The data indicates that, although the regional government sector has the largest overall share of female workers, it has the smallest proportion of female part-time workers. In contrast, the municipal government sector, while having a relatively smaller share of female workers, has a significantly higher percentage, around 35 pct., engaged in part-time employment.

This suggests that the two sectors have different dynamics in the labor market, with the municipal government sector providing more opportunities for part-time work among female employees compared to the regional government sector. It can be inferred that the regional government sector may lean towards employing full-time workers, while the municipal government sector offers greater flexibility in terms of working hours.

# 5. Conclusion

In conclusion, this project examined the gender wage gap and the share of female workers in various sectors in Denmark. By utilizing data from Statistics Denmark, the analysis revealed that the gender wage gap has decreased over time in both sectors, with a significantly smaller gap observed in the municipal government sector. Furthermore, the regional government sector had a higher proportion of female workers, while the municipal government sector had a greater share of female part-time workers.

These findings indicate that the two sectors exhibit distinct dynamics in the labor market, with the municipal government sector providing more opportunities for part-time work among female employees compared to the regional government sector.

Overall, this analysis provides valuable insights into the trends and patterns of the gender wage gap and part-time work in different sectors, contributing to a deeper understanding of the factors influencing these disparities.